In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ocular-data-all-imgs/Combin_tr_te/3419_left.jpg
/kaggle/input/ocular-final-csv/Complete_dataset_forML.csv
/kaggle/input/ocular-disease-recognition-odir5k/full_df.csv
/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images/3419_left.jpg
/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/data.xlsx
/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Testing Images/3552_right.jpg
/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images/3419_left.jpg
/kaggle/input/pseudo-data/glaucoma_hypertensive/623.jpg


In [26]:
import pandas as pd
import numpy as np
import ast
import cv2
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers,Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import Input

from sklearn.model_selection import KFold, StratifiedKFold

from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE,RandomOverSampler
from imblearn.under_sampling import NearMiss, RandomUnderSampler

#import tf_clahe

# Helper Functions and things


In [3]:
def fast_clahe(img):
    return tf_clahe.clahe(img, gpu_optimized=True)
def get_unique_count(col):
    values,counts=np.unique(col,return_counts=True)
    for i,j,k in list(zip(values,counts,counts/sum(counts))):
        print(i,j,round(k,2)) 
@tf.function(experimental_compile=True)  # Enable XLA
def fast_clahe(img):
    return tf_clahe.clahe(img, gpu_optimized=True)

In [4]:
def create_dataset(image_names,labels,clahe=None):
    images=[]
    classes=[]
    for img,label in tqdm(list(zip(image_names,labels))):
        image_path = CFG.images_path+'/'+img
#         try:
        image = cv2.imread(image_path)
        #plt.imshow(image)
        mask = image > 0
        
        # Coordinates of coloured pixels.
        coordinates = np.argwhere(mask)
        # Binding box of non-black pixels.
        x0, y0, s0 = coordinates.min(axis=0)
        x1, y1, s1 = coordinates.max(axis=0) + 1  # slices are exclusive at the top
        #print(x0)
        
        # Get the contents of the bounding box.
        cropped = image[x0:x1, y0:y1]

        #img = tf.image.resize(cropped,CFG.img_size,antialias=True)
        image = cv2.resize(cropped,(CFG.image_size,CFG.image_size))
        #image = cv2.resize(image,(CFG.image_size,CFG.image_size))
#         except:
#             continue

        if clahe:
                try:
                    image = fast_clahe(image)
                    #mage=np.array(image)
                except:
                    print(image)
        dummy=[0]*4
        dummy[label]=1
        images.append(image)
        classes.append(dummy)
    return tf.data.Dataset.from_tensor_slices((images,classes))

In [5]:
class CFG:
    csv_path='/kaggle/input/ocular-final-csv/Complete_dataset_forML.csv'
    images_path='/kaggle/input/ocular-data-all-imgs/Combin_tr_te'
    image_size=512
    n_classes=4

In [6]:
from tensorflow.keras.layers.experimental import preprocessing
trainAug = Sequential([
    #preprocessing.Rescaling(scale=1.0 / 255),
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomFlip("vertical"),
    #preprocessing.RandomRotation(0.9)
])

2022-12-12 18:54:30.827263: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 18:54:30.960082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 18:54:30.960871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 18:54:30.961800: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
def get_metrics(testDS,test_y,model):
    y_preds=model.predict(testDS)
    y_preds_class=[]
    for i in y_preds:
        y_preds_class.append(np.argmax(i))
    y_test_class=[]
    for i in test_y:
        y_test_class.append(np.argmax(i))
    print(classification_report(y_test_class,y_preds_class))

In [8]:
df=pd.read_csv(CFG.csv_path)
# df['Label']= df['Label']-1
# df=df[(df['Label']==0) | (df['Label']==2) | (df['Label']==3) | (df['Label']==5)]
df.head()

,File,Label
0,0_left.jpg,1
1,0_right.jpg,0
2,1_left.jpg,0
3,1_right.jpg,0
4,10_right.jpg,0


In [9]:
get_unique_count(df['Label'])

0 2816 0.77
1 313 0.09
2 326 0.09
3 193 0.05


0-Normal
1- Glacuma
2-cataract
3- Hypertension

In [10]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE,RandomOverSampler
from imblearn.under_sampling import NearMiss, RandomUnderSampler

def sampling_technique(df,test_condition):
    #594 500 397 203
    if test_condition=='Under-Sampling':
        count_class_0 = 300
        count_class_1 = 300
        count_class_2 = 300
        count_class_3 = 300
        
        pipe = make_pipeline(
        RandomUnderSampler(sampling_strategy={0: count_class_0}),
        RandomOverSampler(sampling_strategy={1: count_class_1}),
        RandomOverSampler(sampling_strategy={2: count_class_2}),
        RandomOverSampler(sampling_strategy={3: count_class_3})
    #     RandomOverSampler(sampling_strategy={4: count_class_4}),
    #     RandomOverSampler(sampling_strategy={5: count_class_5}),
    #     RandomOverSampler(sampling_strategy={6: count_class_6}),
    #     RandomOverSampler(sampling_strategy={7: count_class_7})
        )

    if test_condition=='No-Sampling':
        count_class_0 = 900
        count_class_1 = len(df[df['Label']==1])
        count_class_2 = len(df[df['Label']==2])
        count_class_3 = len(df[df['Label']==3])
        
        pipe = make_pipeline(
        RandomUnderSampler(sampling_strategy={0: count_class_0}),
        RandomUnderSampler(sampling_strategy={1: count_class_1}),
        RandomOverSampler(sampling_strategy={2: count_class_2}),
        RandomOverSampler(sampling_strategy={3: count_class_3})
    #     RandomOverSampler(sampling_strategy={4: count_class_4}),
    #     RandomOverSampler(sampling_strategy={5: count_class_5}),
    #     RandomOverSampler(sampling_strategy={6: count_class_6}),
    #     RandomOverSampler(sampling_strategy={7: count_class_7})
        )

    if test_condition=='Over-Sampling':
        count_class_0 = 700
        count_class_1 = 600
        count_class_2 = 600
        count_class_3 = 600

        pipe = make_pipeline(
            RandomUnderSampler(sampling_strategy={0: count_class_0}),
            RandomOverSampler(sampling_strategy={1: count_class_1}),
            RandomOverSampler(sampling_strategy={2: count_class_2}),
            RandomOverSampler(sampling_strategy={3: count_class_3})
        #     RandomOverSampler(sampling_strategy={4: count_class_4}),
        #     RandomOverSampler(sampling_strategy={5: count_class_5}),
        #     RandomOverSampler(sampling_strategy={6: count_class_6}),
        #     RandomOverSampler(sampling_strategy={7: count_class_7})
        )
    X_smt,y_smt = pipe.fit_resample(df, df['Label'])
    return X_smt

In [11]:
df=sampling_technique(df,'No-Sampling')

In [12]:
get_unique_count(df['Label'])

0 900 0.52
1 313 0.18
2 326 0.19
3 193 0.11


# Getting Train, Validation and Test

In [13]:
train, test = train_test_split(df, test_size=0.10, stratify=df['Label'],random_state=1) 
train,val= train_test_split(train, test_size=0.10, random_state=1,stratify=train['Label'])

print("train",len(train))
print("test",len(test))
print("Val",len(val))


train 1402
test 174
Val 156


In [14]:
TrainDS = create_dataset(train['File'].values,train['Label'].values)
valDS = create_dataset(val['File'].values,val['Label'].values)
testDS = create_dataset(test['File'].values,test['Label'].values)

100%|██████████| 1402/1402 [07:05<00:00,  3.29it/s]
2022-12-12 19:01:39.736040: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 8820621312 exceeds 10% of free system memory.
2022-12-12 19:09:14.017325: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1102577664 exceeds 10% of free system memory.
2022-12-12 19:09:16.269995: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1102577664 exceeds 10% of free system memory.
100%|██████████| 156/156 [00:43<00:00,  3.59it/s]
2022-12-12 19:10:00.650307: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 981467136 exceeds 10% of free system memory.
100%|██████████| 174/174 [00:47<00:00,  3.69it/s]
2022-12-12 19:11:38.227066: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1094713344 exceeds 10% of free system memory.


In [55]:
CFG.pseudo_path='/kaggle/input/pseudodata/glaucoma_hypertensive'

In [56]:

pseudoDS = tf.keras.utils.image_dataset_from_directory(
    CFG.pseudo_path,
    labels=None,
    class_names=None,
    color_mode='rgb',
    #batch_size=16,
    image_size=(512, 512),
    shuffle=False,
)

Found 196 files belonging to 1 classes.


In [17]:
# print("train_img_X: ",np.array(train_x).shape)
# print("test_img_X: ",np.array(test_x).shape)
# print("train_labels: ",np.array(train_y).shape)
# print("test_labels: ",np.array(test_y).shape)

In [18]:
#np.array(train_x).shape

In [19]:
trainDS=TrainDS.batch(16).prefetch(tf.data.AUTOTUNE)
valDS=valDS.batch(16).prefetch(tf.data.AUTOTUNE)
testDS=testDS.batch(16).prefetch(tf.data.AUTOTUNE)

# Model Training

In [20]:
checkpoint = ModelCheckpoint('name.h5',monitor="val_acc",verbose=1,save_best_only=True,
                             save_weights_only=False,period=1)

## Different models

In [21]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB7
from tensorflow.keras.applications.xception import Xception

In [22]:
def baseline_model(model):
        vgg = model
        vgg.trainable=False
        input_layer = layers.Input(shape=(CFG.image_size,CFG.image_size,3), name='input') 
        
        ptm = vgg(input_layer)

        l1 = layers.GlobalAveragePooling2D(name='global_average_pooling2d')(ptm)
        l2 = layers.BatchNormalization()(l1)       
        l3 = layers.Dense(128,activation='relu')(l2)

#         l4 = layers.BatchNormalization()(l3)       
#         l5 = layers.Dense(128,activation='relu')(l4)

        
        output = layers.Dense(units=CFG.n_classes, 
                              activation='softmax', 
                              name='output')(l3)
        model = Model (input_layer, output)    
        model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0002), 
                      loss='categorical_crossentropy', 
                      metrics=[ 'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')])
        return model

# efficient_model=baseline_model()
# efficient_model.summary()

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
checkpoint = ModelCheckpoint('name.h5',monitor="val_acc",verbose=1,save_best_only=True,
                             save_weights_only=False,period=1)

In [24]:
#test_y = np.concatenate([y for _, y in testDS], axis=0)

In [27]:
#for efficient net
b0=tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3),input_tensor=Input(shape=(512,512,3)))
#b7=tf.keras.applications.EfficientNetB7(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3))
b4=tf.keras.applications.EfficientNetB4(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3),input_tensor=Input(shape=(512,512,3)))
xception=tf.keras.applications.xception.Xception(include_top=False, weights='imagenet',input_shape=(CFG.image_size,CFG.image_size,3))


83697664/83683744 [==============================] - 4s 0us/step


#### EfficientnetB0

In [50]:
effb0=baseline_model(b0)
effb0.fit(trainDS,batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1,callbacks=[checkpoint])

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/15
88/88 [==============================] - 15s 113ms/step - loss: 0.9873 - accuracy: 0.5949 - precision: 0.6702 - recall: 0.4986 - val_loss: 0.9641 - val_accuracy: 0.6667 - val_precision: 0.8704 - val_recall: 0.3013
Epoch 2/15
88/88 [==============================] - 9s 98ms/step - loss: 0.6941 - accuracy: 0.7268 - precision: 0.7718 - recall: 0.6705 - val_loss: 0.8396 - val_accuracy: 0.7372 - val_precision: 0.8846 - val_recall: 0.4423
Epoch 3/15
88/88 [==============================] - 9s 98ms/step - loss: 0.6030 - accuracy: 0.7575 - precision: 0.7916 - recall: 0.7126 - val_loss: 0.7167 - val_accuracy: 0.7821 - val_precision: 0.8980 - val_recall: 0.5641
Epoch 4/15
88/88 [==============================] - 9s 99ms/step - loss: 0.5617 - accuracy: 0.7846 - precision: 0.8178 - recall: 0.7397 - val_loss: 0.6185 - val_accuracy: 0.7949 - val_precision: 0.8644 - val_recall: 0.6538
Epoch 5/15
88/88 [==============================] - 9s 98ms/step - loss: 0.5126 - accuracy: 0.8017 - preci

In [51]:
test_y = np.concatenate([y for _, y in testDS], axis=0)

In [52]:
test_y[:10]

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]], dtype=int32)

In [53]:
get_metrics(testDS,test_y,effb0)

              precision    recall  f1-score   support

           0       0.80      0.86      0.83        90
           1       0.94      0.91      0.92        32
           2       0.50      0.39      0.44        33
           3       0.57      0.63      0.60        19

    accuracy                           0.75       174
   macro avg       0.70      0.70      0.70       174
weighted avg       0.74      0.75      0.75       174



In [57]:
y_preds=effb0.predict(pseudoDS)
y_preds_ohe=[]
for i in y_preds:
    class_idx=np.argmax(i)
    dummy=[0]*4
    dummy[class_idx]=1
    y_preds_ohe.append(dummy)


In [58]:
pseudoDS.cardinality().numpy()

7

In [59]:
len(y_preds_ohe)

196

In [60]:
pseudo_img = np.concatenate([img for img in pseudoDS], axis=0)

In [61]:
type(pseudo_img)

numpy.ndarray

In [62]:
train_imgs = np.concatenate([img for img,_ in trainDS], axis=0)

In [63]:
type(train_imgs)

numpy.ndarray

In [64]:
pseudo_img=tf.cast(pseudo_img,tf.uint8)
y_preds_ohe=tf.cast(y_preds_ohe,tf.int32)

In [65]:
np.array(y_preds_ohe)[:10]

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]], dtype=int32)

In [66]:
pseudo_with_labels=tf.data.Dataset.from_tensor_slices((np.array(pseudo_img),np.array(y_preds_ohe)))

In [67]:
train_pseduo_DS = TrainDS.concatenate(pseudo_with_labels)

In [ ]:
train_pseudo_DS

In [74]:
y_pseudo = np.concatenate([y for _,y in train_pseduo_DS])
print(len(y_pseudo))

6392


In [73]:
y_pseudo = np.concatenate([y for _,y in trainDS], axis=0)
print(len(y_pseudo))

1402


In [69]:
#train_imgs

# Training using pseudo

In [75]:
effb0_p.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 512, 512, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 16, 16, 1280)      4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 1280)              5120      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               163968    
_________________________________________________________________
output (Dense)               (None, 4)                 516       
Total params: 4,219,175
Trainable params: 167,044
Non-trainable params: 4,052,131
___________________________________________

In [70]:

effb0_p=baseline_model(b0)
effb0_p.fit(train_pseduo_DS.batch(16),batch_size=16,epochs=15,validation_data=(valDS),
                    verbose=1,callbacks=[checkpoint])

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/15
100/100 [==============================] - 16s 110ms/step - loss: 0.8958 - accuracy: 0.6370 - precision: 0.6949 - recall: 0.5544 - val_loss: 0.9704 - val_accuracy: 0.7244 - val_precision: 0.9583 - val_recall: 0.1474
Epoch 2/15
100/100 [==============================] - 10s 99ms/step - loss: 0.6694 - accuracy: 0.7409 - precision: 0.7790 - recall: 0.6840 - val_loss: 0.8149 - val_accuracy: 0.7821 - val_precision: 0.9710 - val_recall: 0.4295
Epoch 3/15
100/100 [==============================] - 10s 97ms/step - loss: 0.5918 - accuracy: 0.7728 - precision: 0.8102 - recall: 0.7240 - val_loss: 0.6658 - val_accuracy: 0.8077 - val_precision: 0.9135 - val_recall: 0.6090
Epoch 4/15
100/100 [==============================] - 10s 97ms/step - loss: 0.5201 - accuracy: 0.8048 - precision: 0.8370 - recall: 0.7678 - val_loss: 0.5763 - val_accuracy: 0.7885 - val_precision: 0.8640 - val_recall: 0.6923
Epoch 5/15
100/100 [==============================] - 10s 97ms/step - loss: 0.4875 - accuracy: 

In [71]:
get_metrics(testDS,test_y,effb0_p)

              precision    recall  f1-score   support

           0       0.81      0.87      0.84        90
           1       0.88      0.94      0.91        32
           2       0.57      0.39      0.46        33
           3       0.57      0.63      0.60        19

    accuracy                           0.76       174
   macro avg       0.71      0.71      0.70       174
weighted avg       0.75      0.76      0.75       174

